In [1]:
import os
from dotenv import load_dotenv

# .env 파일에서 환경 변수를 로드합니다.
load_dotenv()

# os.environ을 사용하여 환경 변수를 가져옵니다.
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY 환경 변수가 설정되어 있지 않습니다.")

print(OPENAI_API_KEY[:10]) # 테스트 후 삭제

sk-svcacct


In [2]:
from typing import Annotated, TypedDict, List, Literal
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

from langchain_openai import ChatOpenAI
# from langchain_core.messages import SystemMessage, AIMessage, HumanMessage, AnyMessage
from langchain_core.tools import tool
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field

In [39]:
import chromadb

client = chromadb.PersistentClient(path="./_data/chroma_db")
collection = client.get_or_create_collection(name="html_template_search_engine")

In [ ]:
# -------------------------------------------------------------
# 1. Pydantic으로 데이터 구조 정의
# -------------------------------------------------------------

BlockType = Literal[
    "Introduction", "KeyFeatures", "Specifications", "UsageGuide", "Comparison", "BrandStory", "FAQ"
]

class StyleConcept(BaseModel):
    """페이지 전체에 적용될 공통 디자인 컨셉입니다."""
    concept_name: str = Field(description="이 스타일 컨셉의 이름 (예: '미니멀 클린', '네온 펑크')")
    color_palette: str = Field(description="페이지의 주요 색상, 배경색, 텍스트 색상")
    font_style: str = Field(description="제목과 본문에 사용할 폰트 스타일")
    overall_mood: str = Field(description="페이지가 전달해야 할 전체적인 분위기")
    css_inspiration: str = Field(description="HTML/CSS 작업 시 참고할 스타일링 가이드")

class ConceptBlock(BaseModel):
    """HTML 생성을 위한 블록 별 스타일 컨셉 및 내용 정보입니다."""
    block_type: BlockType = Field(description="블럭의 시맨틱한 유형")
    content: str = Field(description="""블럭에 들어가야 할 제품의 정보, 내용을 담습니다. 구체적으로 세세하게 작성해야 합니다.""")
    concept_style: str = Field(
        description="""이 블럭의 HTML/CSS 코드를 생성하기 위한 '콘셉트 스타일'을 작성하세요.
        레이아웃, 컴포넌트, 텍스트 톤앤매너, 이미지 배치 등에 대한 콘셉트 스타일을 구체적으로 세세하게 전부 명시해야 합니다. 
        블럭에 들어갈 콘텐츠 내용과 어울려야 합니다."""
    )

class ProductPage(BaseModel):
    """상품 상세 페이지 전체 구조입니다."""
    style_concept: StyleConcept = Field(description="페이지 전체의 일관된 스타일 가이드")
    concept_blocks: List[ConceptBlock] = Field(
        description="상품 상세 페이지의 콘셉 스타일을 구성하는 블럭 리스트",
        min_items=1,
    )


In [46]:
@tool
def get_block_template(block_type: BlockType, concept_style: str) -> List[str]:
    """
    지정된 블록 타입과 관련 콘셉트 스타일을 기반으로
    상품 상세 페이지의 특정 블록에 대한 설계도를 생성합니다.

    Args:
        block_type: 생성할 블록의 유형입니다.
        concept_style: 블록에 어울리는 콘셉트 스타일입니다.

    Return: concept_style 과 관련된 html 코드 템플릿을 3개 가져옵니다.
    """

    results = collection.query(
        query_texts=[concept_style],
        where={
            "$and": [
                {"block_type":block_type},
                {"category":"생활용품"}
            ]
        },
        n_results=3
    )

    templates = [x['template'] for x in results['metadatas'][0]]
    return templates
    

In [55]:
# -------------------------------------------------------------
# 2. LangChain 및 모델 설정
# -------------------------------------------------------------

def generate_product_page_layout(product_info: str) -> ProductPage:
    """
    상품 정보를 분석하여, HTML 생성을 위한 페이지 '설계도'를 생성합니다.
    (공통 스타일 컨셉 + 각 블럭별 통합 코딩 지시서)
    """
    llm = ChatOpenAI(
        model="gpt-4o-mini",
        temperature=0.2,
        api_key=os.getenv("OPENAI_API_KEY")
    )
    structured_llm = llm.with_structured_output(ProductPage)

    # 이미지 지시사항을 통합하고, 특별 태그 사용을 명시, LLM의 역할을 명확히 하고, 금지 조항과 예시를 추가하여 제어 강화
    system_prompt = """
    당신은 개발자를 위한 최종적이고 상세한 청사진을 만드는 세계적인 아트 디렉터입니다.
    출력물은 `style_concept`와 `ConceptBlock` 목록을 포함하는 Pydantic 객체여야 합니다.
    각 `ConceptBlock` 내부의 `concept_style`, `content`는 반드시 상세한 내용으로 구성된 단일 문자열이어야 합니다.

    **핵심 지침**
    1. **크리에이티브 디렉터가 되세요:** 각 블록에 대해 구체적이고 창의적이며 적절한 시각적 스타일을 고안하세요. 모든 것에 하나의 스타일을 사용하지 마세요. 블록의 목적을 생각하고 그에 따라 디자인하세요.
    2. **선택적 생성:** `product_info`에 충분한 정보가 있는 경우에만 블록을 생성하세요. 콘텐츠를 새로 만들지 마세요.
    3. **동일한 여러 블록타입 사용(선택):** 블록타입은 중복하여 사용 가능합니다. 긴 내용을 하나의 블록에 전부 담지 말고 분할하여 만드세요.
    3. **한국어 사용:** 모든 언어는 한국어를 사용해야합니다.
    """
    # **--- `html_generation_prompt`의 핵심 지침 ---**

    # 1. **크리에이티브 디렉터가 되세요:** 각 블록에 대해 구체적이고 창의적이며 적절한 시각적 스타일을 고안하세요. 모든 것에 하나의 스타일을 사용하지 마세요. 블록의 목적을 생각하고 그에 따라 디자인하세요.
    # 2. **매우 구체적으로 작성하세요:** 추상적인 용어는 사용하지 마세요.
    # - 나쁨: "현대적으로 보이도록 하세요."
    # - 좋음: "`justify-content: space-between`을 적용한 플렉스박스 레이아웃을 사용하세요. 간결하고 깔끔한 느낌을 위해 메인 헤드라인은 `letter-spacing: -0.02em`을 적용해야 합니다."
    # 3. **이중 언어 콘텐츠:** 모든 지침은 반드시 영어로 작성해야 합니다. 모든 고객 대상 텍스트(헤드라인, 단락)는 반드시 한국어로 작성해야 합니다.
    # 4. **정적 이미지만 사용:** 최종 결과물은 정적 이미지입니다. 애니메이션이나 호버 효과를 포함해서는 안 됩니다.
    # 5. **이미지 태그:** `[DALLE_IMAGE: ...]` 또는 `[PRODUCT_IMAGE: ...]`를 사용하여 이미지 배치 방법을 안내하세요.
    # 6. **선택적 생성:** `product_info`에 충분한 정보가 있는 경우에만 블록을 생성하세요. 콘텐츠를 새로 만들지 마세요.
    # """

    human_prompt = "{product_info}"
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", human_prompt)
    ])

    chain = prompt | structured_llm
    return chain.invoke({"product_info": product_info})

In [56]:
detailed_product_info = """
AURA Smart Mood Speaker (모델명: AURA-7)
"당신의 공간과 감정을 빛과 소리로 디자인합니다."

AURA는 단순한 스피커가 아닙니다. 사용자의 목소리 톤, 대화 내용, 시간, 그리고 연결된 캘린더의 일정을 실시간으로 분석하여 현재 감정 상태에 가장 적합한 음악과 조명을 자동으로 큐레이션하는 인공지능 감성 조명 스피커입니다. 바쁜 아침에는 활기찬 음악과 태양광과 유사한 조명으로 활력을, 고요한 저녁에는 차분한 연주곡과 촛불처럼 은은한 조명으로 평온을 선사합니다.

## 주요 특징 (Key Features)
실시간 감성 분석 AI, 'Lumi': AURA의 핵심인 'Lumi' AI는 4개의 고감도 마이크 어레이를 통해 사용자의 음성을 분석하고, 미세한 감정 변화를 감지합니다. "오늘 좀 피곤하네"라는 혼잣말만으로도, AURA는 재충전에 도움이 되는 사운드와 조명 테라피를 제안합니다.

360° 몰입형 공간 음향: 독일의 명품 오디오 브랜드 'Klang & Stille'과의 협업으로 탄생한 3개의 풀레인지 드라이버와 1개의 듀얼 패시브 라디에이터가 360° 모든 방향으로 균일하고 풍부한 사운드를 전달합니다. 청취자의 위치를 파악해 실시간으로 사운드를 최적화하는 '어댑티브 EQ' 기술이 탑재되어 있습니다.

1,600만 컬러 다이나믹 앰비언트 라이트: 음악의 비트와 리듬에 맞춰 춤추는 라이트 쇼부터, 실제 일출과 일몰의 색 변화를 그대로 재현하는 '서카디안 리듬 모드'까지 지원합니다. 아침에는 부드러운 주황빛으로 시작해 정오에는 밝은 백색광으로, 저녁에는 따뜻한 색감으로 자동 변경되어 건강한 생체리듬 유지를 돕습니다.

차세대 스마트홈 허브: 최신 통신 표준인 Matter와 Thread를 동시 지원하여, 집 안의 다른 스마트 기기들을 위한 강력하고 안정적인 허브 역할을 수행합니다. 조명, 온도 조절기, 블라인드 등을 AURA에게 음성으로 손쉽게 제어할 수 있습니다.

개인정보 보호 최우선 설계: 모든 음성 데이터는 기기 내부의 'AURA Hexa-Core NPU' 칩셋에서 처리되는 것을 원칙으로 합니다. 또한, 원터치로 마이크를 물리적으로 차단하는 프라이버시 스위치가 탑재되어 있습니다.

## 기술 사양 (Technical Specifications)
오디오

드라이버: 1.5인치 풀레인지 드라이버 x 3, 듀얼 포스 캔슬링 패시브 라디에이터 x 1

주파수 응답: 55Hz - 22,000Hz

오디오 기술: 360° 공간 음향, 어댑티브 EQ, Dolby Atmos 지원

조명

LED: 90개의 커스텀 RGBW LED

색상 표현: 1,600만 색상

최대 밝기: 800 루멘

연결성

Wi-Fi: Wi-Fi 6E (802.11ax)

Bluetooth: Bluetooth 5.3

스마트홈 프로토콜: Matter, Thread, Zigbee

AI 및 센서

프로세서: AURA Hexa-Core NPU (On-Device AI)

마이크: 4-mic Far-field Array (빔포밍 기술 적용)

센서: 온도 센서, 습도 센서, 조도 센서

규격

크기: 180mm (높이) x 140mm (직경)

무게: 1.2 kg

소재: 재활용 알루미늄, Kvadrat 어쿠스틱 패브릭

## 구성품 및 기타
구성품: AURA-7 본체, 30W USB-C 전원 어댑터, 1.5m 패브릭 USB-C 케이블, 빠른 시작 가이드

색상 옵션: 미드나잇 애쉬 (Midnight Ash), 글레이셔 실버 (Glacier Silver)

가격 및 출시 정보: 349,000원 (2025년 9월 1일 출시 예정)

타겟 고객:

최신 기술과 디자인에 민감한 테크 얼리어답터

음악 감상과 인테리어를 중요하게 생각하는 20-40대

명상, 요가 등 웰니스 활동을 즐기며 마음의 안정을 추구하는 사용자

## 자주 묻는 질문 (FAQ)
Q: AURA 구독 서비스가 필요한가요?

A: 기본적인 AI 음악/조명 큐레이션 및 스마트홈 기능은 모두 무료입니다. 다만, 전문 심리상담사와 협업하여 제작된 '프리미엄 마인드풀니스' 콘텐츠(가이드 명상, 집중력 향상 사운드 등)는 월 4,900원의 구독이 필요합니다.

Q: 기존에 사용하던 스포티파이, 유튜브 뮤직과 연동되나요?

A: 네, 스포티파이 커넥트와 구글 캐스트를 모두 지원하여 기존에 사용하시던 앱에서 손쉽게 음악을 재생할 수 있습니다.

Q: 조명만 끄고 음악만 들을 수 있나요?

A: 네, "오라, 조명 꺼줘" 또는 앱을 통해 언제든지 조명과 음악을 개별적으로 제어할 수 있습니다.
"""

page_layout = generate_product_page_layout(detailed_product_info)

In [57]:
display(page_layout.style_concept)
display(page_layout.concept_blocks)

StyleConcept(concept_name='감성 조화', color_palette='부드러운 파스텔 톤과 자연을 연상시키는 색상 조합 (민트 그린, 라벤더, 따뜻한 베이지)', font_style='모던하고 세련된 산세리프체, 부드러운 곡선이 강조된 글꼴', overall_mood='편안하고 아늑한 분위기, 감정의 흐름을 느낄 수 있는 따뜻한 느낌', css_inspiration='미니멀리즘과 자연 요소를 결합한 디자인, 부드러운 그라디언트 배경과 간결한 레이아웃')

[ConceptBlock(block_type='Introduction', content='AURA Smart Mood Speaker는 단순한 스피커가 아닌, 사용자의 감정 상태를 분석하여 최적의 음악과 조명을 제공하는 인공지능 감성 조명 스피커입니다. 바쁜 아침에는 활기찬 음악과 태양광과 유사한 조명으로 활력을, 고요한 저녁에는 차분한 연주곡과 은은한 조명으로 평온을 선사합니다.', concept_style='부드러운 파스텔 색상과 자연을 연상시키는 디자인 요소를 활용하여, 감정의 흐름을 시각적으로 표현합니다. 사용자 경험을 강조하는 직관적인 레이아웃과 함께, 따뜻한 느낌의 폰트를 사용하여 친근감을 더합니다.'),
 ConceptBlock(block_type='KeyFeatures', content='1. 실시간 감성 분석 AI, \'Lumi\': AURA의 핵심인 \'Lumi\' AI는 4개의 고감도 마이크 어레이를 통해 사용자의 음성을 분석하고, 미세한 감정 변화를 감지합니다. "오늘 좀 피곤하네"라는 혼잣말만으로도, AURA는 재충전에 도움이 되는 사운드와 조명 테라피를 제안합니다.\n\n2. 360° 몰입형 공간 음향: 독일의 명품 오디오 브랜드 \'Klang & Stille\'과의 협업으로 탄생한 3개의 풀레인지 드라이버와 1개의 듀얼 패시브 라디에이터가 360° 모든 방향으로 균일하고 풍부한 사운드를 전달합니다. 청취자의 위치를 파악해 실시간으로 사운드를 최적화하는 \'어댑티브 EQ\' 기술이 탑재되어 있습니다.\n\n3. 1,600만 컬러 다이나믹 앰비언트 라이트: 음악의 비트와 리듬에 맞춰 춤추는 라이트 쇼부터, 실제 일출과 일몰의 색 변화를 그대로 재현하는 \'서카디안 리듬 모드\'까지 지원합니다. 아침에는 부드러운 주황빛으로 시작해 정오에는 밝은 백색광으로, 저녁에는 따뜻한 색감으로 자동 변경되어 건강한 생체리듬 유지를 돕습니다.\n\n4. 차세대 스마트홈 허브: 최신 통신 표준인 Matter와 Thread를 동시 지원하여, 집 안의 다른 스

In [127]:
def markdown_to_html(markdown_text: str) -> str:
    """
    마크다운 형식의 코드 블럭 문자열에서 순수한 HTML 코드만 추출합니다.
    """
    # 앞뒤의 불필요한 공백이나 개행 문자를 먼저 제거합니다.
    clean_text = markdown_text.strip()
    
    # '```html'로 시작하는 경우, 해당 부분을 제거합니다.
    if clean_text.startswith("```html"):
        clean_text = clean_text[7:]
        
    # '```'로 끝나는 경우, 해당 부분을 제거합니다.
    if clean_text.endswith("```"):
        clean_text = clean_text[:-3]

    clean_text = clean_text.replace("\n", "")
    return clean_text.strip()

In [133]:
# -------------------------------------------------------------
# 4. 템플릿 사용하여 블록 html 만들기
# -------------------------------------------------------------

def create_html_block(block: any, style: StyleConcept) -> str:
    enhancer_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3, api_key=os.getenv("OPENAI_API_KEY"))
    system_prompt = """
    당신은 세계 최고 수준의 아트 디렉터입니다. 당신의 임무는 html 템플릿들과 유사도 정보, 들어가야 할 내용을 받아, 블럭의 컨셉에 맞는 독창적이고 상세한 시각적 html 블럭를 완성하는 것입니다.
    
    **핵심 가이드 원칙**

    1.  **컨셉의 시각화:** 블럭이 전달하고자 하는 핵심 메시지(예: '혁신적인 기술', '편안함', '신뢰도')를 내용을 보고 시각적으로 어떻게 표현할지 깊이 고민하세요. 템플릿과 공통 컨셉 스타일을 참고하여 색상, 레이아웃, 타이포그래피, 여백 등을 통해 해당 컨셉을 구체화해야 합니다.
    2.  **정적인 임팩트:** 최종 결과물은 정적인 이미지입니다. 따라서 움직임(애니메이션, 호버 효과) 없이도 시선을 사로잡고 정보가 명확히 전달될 수 있는 강렬한 시각적 디자인을 구상해야 합니다.
    
    **당신의 작업 지시**

    기본 템플릿와 전체 스타일 컨셉을 바탕으로 블록 html 코드를 생성하세요. 당신의 목표는 아래 원칙에 따라 전문가 웹디자이너 수준의 html 코드로 만드는 것입니다.

    1.  **독창적 스타일 제안:** `style_concept`을 참고하되, 블럭의 `block_type`과 내용에 가장 잘 어울리는 구체적인 디자인 스타일(예: 미니멀리즘, 타이포그래피 중심, 인포그래픽 스타일 등)을 스스로 생각하세요.
    2.  **카피와 디자인의 조화:** 설득력 있게 다듬은 한글 카피가 디자인 요소와 어떻게 어우러져야 하는지 생각하세요.
    3.  **최종 출력:** 위 모든 요소를 종합하여, 당신의 창의적인 생각을 html 코드로 즉시 변환하세요.

    **주의**
    최종 출력은 반드시 html 코드만을 반환해야 합니다.
    """

    human_prompt_template = """
    **전반적인 스타일 컨셉을 따르세요:**
    {style_concept}
    **---**
    **기본 템플릿과 들어가야 할 내용:**
    {template_info}
    """
    prompt = ChatPromptTemplate.from_messages([("system", system_prompt), ("human", human_prompt_template)])
    chain = prompt | enhancer_llm | StrOutputParser()
    
    html = chain.invoke({
        "style_concept": style.model_dump_json(indent=2),
        "template_info": block,
    })
    
    # 강화된 프롬프트로 새로운 블럭 객체를 만들어 반환
    return markdown_to_html(html)

# -------------------------------------------------------------
# 3. 콘셉트 html 템플릿 가져오기
# -------------------------------------------------------------

def enhance_blueprint_details(product_page: ProductPage) -> any:
    style_concept = product_page.style_concept
    concept_blocks = product_page.concept_blocks
    results = []

    for block in concept_blocks:
        print('create block..', block.content)
        search_results = collection.query(
            query_texts=[block.concept_style],
            where={
                "$and": [
                    {"block_type":block.block_type},
                    {"category":"생활용품"} # 일단 고정
                ]
            },
            n_results=3
        )

        distances = search_results['distances'][0]
        templates = [x['template'] for x in search_results['metadatas'][0]]
        
        results.append({ 
            "template": [{ "distance": distances[i], "html": templates[i] } for i, _ in enumerate(distances)],
            "content": block.content
        })

    html_results = []

    for result in results:
        print('create html..', result.content)
        html_results.append(create_html_block(result, style_concept))

    return html_results
    



In [130]:
results = enhance_blueprint_details(page_layout)

부드러운 파스텔 색상과 자연을 연상시키는 디자인 요소를 활용하여, 감정의 흐름을 시각적으로 표현합니다. 사용자 경험을 강조하는 직관적인 레이아웃과 함께, 따뜻한 느낌의 폰트를 사용하여 친근감을 더합니다. Introduction
각 기능의 독창성을 강조하기 위해, 각 기능에 맞는 아이콘과 그래픽 요소를 사용하여 시각적으로 매력적인 레이아웃을 구성합니다. 색상은 부드러운 톤을 유지하며, 사용자 친화적인 인터페이스를 제공합니다. KeyFeatures
기술 사양을 명확하게 전달하기 위해, 표 형식으로 정리하여 가독성을 높입니다. 각 사양 항목은 아이콘과 함께 시각적으로 구분하여 사용자가 쉽게 이해할 수 있도록 디자인합니다. Specifications
사용자 가이드는 단계별로 명확하게 설명하여, 사용자가 쉽게 따라할 수 있도록 돕습니다. 각 단계는 일러스트와 함께 제공하여 시각적인 이해를 돕습니다. UsageGuide
자주 묻는 질문 섹션은 명확하고 간결하게 정보를 제공하여 사용자가 궁금증을 쉽게 해결할 수 있도록 합니다. 각 질문은 강조된 제목과 함께 제공하여 가독성을 높입니다. FAQ


In [132]:
results

['<div style="background: linear-gradient(to bottom, #d1e8e2, #f3e5f5); text-align: center; padding: 60px 20px; position: relative;">    <img src="[부드러운 흰색 원형 오브제들]" style="position: absolute; top: 0; left: 0; width: 100%; height: 100%; opacity: 0.5;">    <div style="position: relative;">        <h2 style="font-size: 28px; font-weight: bold; color: #5b8c7a; margin-top: 20px; font-family: \'Noto Sans KR\', sans-serif;">AURA Smart Mood Speaker</h2>        <p style="font-size: 16px; color: #333; margin-top: 10px; font-family: \'Helvetica\', sans-serif;">단순한 스피커가 아닌, 사용자의 감정 상태를 분석하여 최적의 음악과 조명을 제공합니다.</p>        <img src="[AURA Smart Mood Speaker 이미지]" style="width: 60%; margin-top: 20px; border-radius: 15px; box-shadow: 0 4px 10px rgba(0, 0, 0, 0.1);">        <div style="margin-top: 30px; padding: 20px; background-color: rgba(255, 255, 255, 0.8); border-radius: 10px;">            <h3 style="font-size: 22px; color: #5b8c7a; font-weight: bold; margin: 0;">바쁜 아침에는</h3>            <p style="

In [138]:
print(results[4])

<div style="background-color: #f9f9f9; padding: 30px; border-radius: 8px; box-shadow: 0 2px 10px rgba(0, 0, 0, 0.1);">    <div style="text-align: center; margin-bottom: 20px;">        <img src="[미소라이프 고객센터 로고]" style="height: 50px; margin-bottom: 10px;">        <p style="font-size: 14px; color: #555; margin-top: 5px;">070-4335-7823으로 문자 문의 부탁드립니다.</p>    </div>    <div style="border-top: 1px dashed #ccc; border-bottom: 1px dashed #ccc; padding: 15px 0; margin-bottom: 15px;">        <h4 style="font-size: 16px; font-weight: bold; color: #e74c3c;">업무/답안안내</h4>        <p style="font-size: 14px; color: #666;">고객센터 10:00~17:00 (점심시간 12:00~01:00)</p>    </div>    <div style="border-bottom: 1px dashed #ccc; padding-bottom: 15px; margin-bottom: 15px;">        <h4 style="font-size: 16px; font-weight: bold; color: #3498db;">배송안내</h4>        <p style="font-size: 14px; color: #666;">택배는 결제 완료 후 1~3일 안에 배송됩니다. 당일출고 마감시간은 오후 02:00까지 주문건에 대하여 발송됩니다. CJ대한통운을 이용합니다.</p>    </div>    <div style="border-b

In [31]:
result = generate_html_for_block(product_page_layout_simple.blocks[0], product_page_layout_simple.style_concept)

In [32]:
result

'```html\n<section>\n    <style>\n        body {\n            font-family: \'Roboto\', sans-serif;\n            background-color: #F0E68C;\n            margin: 0;\n            padding: 20px;\n        }\n        .key-features {\n            display: grid;\n            grid-template-columns: repeat(3, 1fr);\n            gap: 24px;\n        }\n        .feature-card {\n            border: 1px solid #EAECEF;\n            border-radius: 16px;\n            background-color: white;\n            padding: 20px;\n            text-align: center;\n            box-shadow: 0 2px 10px rgba(0, 0, 0, 0.1);\n        }\n        h3 {\n            color: #B2DFF7;\n        }\n        p {\n            color: #D1C6E7;\n        }\n    </style>\n    <div class="key-features">\n        <div class="feature-card">\n            <img src="[DALLE_IMAGE: minimalist line icon for 3-speed wind control]" alt="3-Speed Wind Control Icon">\n            <h3>3-Speed Wind Control</h3>\n            <p>Adjust the airflow to your 